<a href="https://colab.research.google.com/github/1221nana7/Basic-Python/blob/pandas/pandas_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものです。

## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [3]:
import pandas as pd
import numpy as np

## データの読み込み

In [4]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [5]:
df_receipt.loc[:,['sales_ymd','customer_id','product_cd','quantity','amount']].query('customer_id=="CS018205000001" & (amount >=1000 | quantity >= 5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [6]:
df_customer.sort_values(by='birth_day', ascending=False).head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [18]:
df_receipt.count().values[1]

104681

## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [19]:
df_receipt['customer_id'].nunique()

8307

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [9]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [22]:
amount_customer = df_receipt_only_member[['customer_id','amount']].groupby('customer_id').sum()
average_customer = amount_customer['amount'].mean()
over_average_customer = df_receipt_only_member[df_receipt_only_member['amount']>= average_customer]
over_average_customer.head(5)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
89,20190615,1560556800,S13039,1102,2,CS039415000129,P090404061,1,2580
596,20181219,1545177600,S13015,1132,1,CS015515000083,P071301009,1,5280
1202,20190903,1567468800,S14011,1192,2,CS011415000006,P080104055,1,10925
2715,20180825,1535155200,S13035,1142,1,CS035415000125,P071301006,1,4760
2950,20180809,1533772800,S14023,1192,1,CS023513000144,P071401001,3,3300


---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [11]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [72]:
df_customer_amount = df_receipt.groupby('customer_id').amount.sum()
df_woman_member = df_customer.query('gender_cd ==1 & not customer_id.str.startswith("Z")',engine='python')
df_customer_merge = pd.merge(df_woman_member['customer_id'],df_customer_amount,how='left',on='customer_id')
df_customer_merge.fillna(0).head(5)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0


---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [79]:
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format = '%Y%m%d')
df_customer.loc[:,['customer_id', 'application_date']].head(5)

,customer_id,application_date
0,CS021313000114,1970-01-01 00:00:00.020150905
1,CS037613000071,1970-01-01 00:00:00.020150414
2,CS031415000172,1970-01-01 00:00:00.020150529
3,CS028811000001,1970-01-01 00:00:00.020160115
4,CS001215000145,1970-01-01 00:00:00.020170605


---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [68]:
df_customer_sum = df_receipt.groupby(df_receipt_only_member['customer_id']).sum()
df_customer_amount = df_customer_sum.loc[:,['amount']]
df_customer_amount['log_amount'] = np.log10(df_customer_amount['amount']+1)
df_customer_amount.head(5)

,amount,log_amount
customer_id,,
CS001113000004,1298,3.113609
CS001114000005,626,2.797268
CS001115000010,3044,3.483587
CS001205000004,1988,3.298635
CS001205000006,3337,3.523486


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [23]:
df_product.isnull().sum()
#df_product.isna()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [26]:
df_product_1 = df_product.dropna()

In [28]:
print('削除前:',len(df_product)) 
print('削除後:',len(df_product_1))

削除前: 10030
削除後: 10023


---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [62]:
df_product_2 = df_product
price_mean = df_product_2['unit_price'].mean().round()
cost_mean = df_product_2['unit_cost'].mean().round()
df_product_2['unit_price'].fillna(price_mean)
df_product_2['unit_cost'].fillna(cost_mean)
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64